In [1]:
import sys
sys.path.append("..")

In [2]:
from collections import Counter, defaultdict
from enum import Enum

from resources.utils import get_puzzle_input

In [3]:
def adjacent(point):
    offsets = (
        (0, 1),
        (1, 1),
        (1, 0),
        (1, -1),
        (0, -1),
        (-1, -1),
        (-1, 0),
        (-1, 1)
    )
    return [(point[0] + x, point[1] + y) for (x, y) in offsets]

def direction_move(pos, direction):
    offset_x, offset_y = MOVE_MAP[direction]
    return pos[0] + offset_x, pos[1] + offset_y

### Part 1


On the outskirts of the North Pole base construction project, many Elves are collecting lumber.

The lumber collection area is 50 acres by 50 acres; each acre can be either open ground (.), trees (|), or a lumberyard (#). You take a scan of the area (your puzzle input).

Strange magic is at work here: each minute, the landscape looks entirely different. In exactly one minute, an open acre can fill with trees, a wooded acre can be converted to a lumberyard, or a lumberyard can be cleared to open ground (the lumber having been sent to other projects).

The change to each acre is based entirely on the contents of that acre as well as the number of open, wooded, or lumberyard acres adjacent to it at the start of each minute. Here, "adjacent" means any of the eight acres surrounding that acre. (Acres on the edges of the lumber collection area might have fewer than eight adjacent acres; the missing acres aren't counted.)

In particular:

* An open acre will become filled with trees if three or more adjacent acres contained trees. Otherwise, nothing happens.
* An acre filled with trees will become a lumberyard if three or more adjacent acres were lumberyards. Otherwise, nothing happens.
* An acre containing a lumberyard will remain a lumberyard if it was adjacent to at least one other lumberyard and at least one acre containing trees. Otherwise, it becomes open.
* These changes happen across all acres simultaneously, each of them using the state of all acres at the beginning of the minute and changing to their new form by the end of that same minute. Changes that happen during the minute don't affect each other.

In [4]:
class Arena:
    def __init__(self, initial_state):
        self.state = dict(initial_state)
        self.initial_state = dict(initial_state)
        self.max_x, self.max_y = self._dims()
    
    def _dims(self):
        max_x = None
        max_y = None
        for x, y in self.state:
            if max_x is None or x > max_x:
                max_x = x
            if max_y is None or y > max_y:
                max_y = y

        return max_x, max_y
    
    def move(self):
        new_state = dict(self.state)
        
        for x in range (0, self.max_x + 1):
            for y in range (0, self.max_y + 1):
                point = x, y
                neighbours = self.neighbours(point)
                my_state = self.state[point]
                
                # An open acre will become filled with trees if three or more
                # adjacent acres contained trees. Otherwise, nothing happens.
                if my_state == '.' and neighbours['|'] >= 3:
                    new_state[point] = '|'
                
                # An acre filled with trees will become a lumberyard if three or more
                # adjacent acres were lumberyards. Otherwise, nothing happens.
                elif my_state == '|' and neighbours['#'] >= 3:
                    new_state[point] = '#'

                # An acre containing a lumberyard will remain a lumberyard if it was
                # adjacent to at least one other lumberyard and at least one acre containing trees.
                # Otherwise, it becomes open.
                elif my_state == '#' and (neighbours['#'] < 1 or neighbours['|'] < 1):
                    new_state[point] = '.'
                 
                # Otherwise, nothing happens.
                else:
                    new_state[point] = my_state
                    
        self.state = new_state
        
    def count(self):
        counter = Counter()
        for x in range (0, self.max_x + 1):
            for y in range (0, self.max_y + 1):
                point = x, y
                counter[self.state[point]] += 1
                
        return counter
        
    def neighbours(self, point):
        neighbours_found = Counter()
        
        for neighbour in adjacent(point):
            if neighbour in self.state:
                neighbours_found[self.state[neighbour]] += 1
        return neighbours_found

    def __hash__(self):
        return hash(tuple(sorted(self.state.items())))

    def __repr__(self):
        lines = []
        for y in range (0, self.max_y + 1):
            line = []
            for x in range (0, self.max_x + 1):
                char = self.state.get((x, y), '.')
                line.append(char)
            lines.append(''.join(line))
        
        return '\n'.join(lines)

In [5]:
def parse_input(lines):
    y = 0
    grid = {}
    for line in lines:
        x = 0
        for char in line:
            grid[x, y] = char
            x += 1        
        y += 1
    return Arena(grid)

In [6]:
test_input = """.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.
""".split('\n')

In [7]:
test_grid = parse_input(test_input)

In [8]:
test_grid = parse_input(test_input)
print(test_grid)
for i in range(1, 11):
    test_grid.move()
    print()
    print('=======')
    print('Move', i)
    print(test_grid)
    
test_grid.count()

.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.

Move 1
.......##.
......|###
.|..|...#.
..|#||...#
..##||.|#|
...#||||..
||...|||..
|||||.||.|
||||||||||
....||..|.

Move 2
.......#..
......|#..
.|.|||....
..##|||..#
..###|||#|
...#|||||.
|||||||||.
||||||||||
||||||||||
.|||||||||

Move 3
.......#..
....|||#..
.|.||||...
..###|||.#
...##|||#|
.||##|||||
||||||||||
||||||||||
||||||||||
||||||||||

Move 4
.....|.#..
...||||#..
.|.#||||..
..###||||#
...###||#|
|||##|||||
||||||||||
||||||||||
||||||||||
||||||||||

Move 5
....|||#..
...||||#..
.|.##||||.
..####|||#
.|.###||#|
|||###||||
||||||||||
||||||||||
||||||||||
||||||||||

Move 6
...||||#..
...||||#..
.|.###|||.
..#.##|||#
|||#.##|#|
|||###||||
||||#|||||
||||||||||
||||||||||
||||||||||

Move 7
...||||#..
..||#|##..
.|.####||.
||#..##||#
||##.##|#|
|||####|||
|||###||||
||||||||||
||||||||||
||||||||||

Move 8
..||||##..
..|#####..
|||#####|.
||#...##|#
||##..###|
|

Counter({'.': 32, '|': 37, '#': 31})

In [9]:
counter = test_grid.count()
counter['|'] * counter['#']

1147

In [10]:
puzzle_input = get_puzzle_input('/tmp/day_18.txt')

In [11]:
puzzle_grid = parse_input(puzzle_input)
for i in range(10):
    puzzle_grid.move()
    
counter = puzzle_grid.count()
counter['|'] * counter['#']

560091

### Part 2

This important natural resource will need to last for at least thousands of years. Are the Elves collecting this lumber sustainably?

What will the total resource value of the lumber collection area be after 1000000000 minutes?

In [12]:
prev_states = set()
state_hist = []
puzzle_grid = parse_input(puzzle_input)
while True:
    state = hash(puzzle_grid)
    state_hist.append(state)
    if state in prev_states:
        break
    prev_states.add(state)
    puzzle_grid.move()

In [13]:
seen_state = state_hist[-1]
state_hist.index(seen_state)

443

In [14]:
len(state_hist)

472

In [15]:
state_hist[471], state_hist[443]

(-326903804210044452, -326903804210044452)

In [17]:
target_index = 443 + ((1000000000 - 443) % (471 - 443))
target_index

468

In [18]:
# Double check they are the same
puzzle_grid = parse_input(puzzle_input)
for i in range(443):
    puzzle_grid.move()
    
hash(puzzle_grid), puzzle_grid.count()

(-326903804210044452, Counter({'.': 1582, '#': 333, '|': 585}))

In [19]:
puzzle_grid = parse_input(puzzle_input)
for i in range(471):
    puzzle_grid.move()
    
hash(puzzle_grid), puzzle_grid.count()

(-326903804210044452, Counter({'.': 1582, '#': 333, '|': 585}))

In [20]:
puzzle_grid = parse_input(puzzle_input)
for i in range(target_index):
    puzzle_grid.move()
    
counter = puzzle_grid.count()
counter['|'] * counter['#']

202301